In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact

import resourcecode
from resourcecode.nataf import censgaussfit, get_gpd_parameters, run_simulation, huseby

In [ ]:
client = resourcecode.Client()
data = client.get_dataframe_from_criteria(
    """
{
    "node": 0,
    "start": 961939610,
    "end": 1489903600,
    "parameter": ["hs", "ucur", "vcur"]
}
"""
)

In [ ]:
@interact(quant=(0.5, 0.9, 0.1))
def plot_nataf(quant):
    gauss = censgaussfit(data.values, quant).x
    gpd_parameters = get_gpd_parameters(data, quant)
    nataf = run_simulation(gauss, quant, gpd_parameters)

    X, Y, Z, theta = huseby(nataf, np.array([0.9, 0.95, 0.975]), 120)

    fig = go.Figure()

    fig.add_traces(
        [
            go.Scatter3d(
                x=X[:, i],
                y=Y[:, i],
                z=Z[:, i],
                mode="lines",
                name=data.columns[i],
            )
            for i, name in enumerate(data.columns)
        ]
    )

    fig.update_layout(
        height=1200,
    )

    return fig